# NRGV Data Analyst Interview Project

## Data Processing

## Objectives 

1) Identify the best location to install a solar array considering the following states:<br>
<ul>
    <li>California</li>
    <li>Texas</li>
    <li>Alabama</li>
    <li>Florida</li>
    <li>Maine</li>
    <li>Colorado</li>
</ul>
2) Present your selected location using graphics that best convey your results for senior leadership<br>
3) For the location you have identified, calculate the forecasted power that a solar array would produce in megawatts<br>
<ul>
    <li>Present your calculations in a graph that shows production monthly for a year</li>
    <li>What other graphics can provide insight into the forecasted power production?</li>
</ul>
4) Extra credit ideas (you can be creative and find other ways to analyze this data):<br>
<ul>
    <li>Plot the candidate sites on a map with an overlay of their forecasted production</li>
    <li>Automate the download, unziping and ingest of study data to increase the number of states you analyze</li>
    <li>What impact did weather have on power production rates?</li>
</ul>
Data: https://www.nrel.gov/grid/solar-power-data.html

### Future Work - Ideas

1) Could test for count of underscores in file name to chack for additional encodings<br>
2) Using LAT/LONG and Date, determine sunset/sunrise; progamaticaly remove records when the sun is not out (from sunset to sunrise).<br>
2a) Eliminate these records during the injest stage -Document time-delta vs lines of code vs reduced records.<br>
3) Automate the detection of new state in dictionary.

### Assumptions 

<ul>
    <li>NREL's Solar Power Data for Integration Studies are synthetic solar photovoltaic (PV) power plant data points for the United States representing the year 2006.</li>
    <li>NREL The data are intended for use by energy professionals—such as transmission planners, utility planners, project developers, and university researchers—who perform solar integration studies and need to estimate power production from hypothetical solar plants.</li>
    <li> The Solar Power Data for Integration Studies consist of 1 year (2006) of 5-minute solar power and hourly day-ahead forecasts for approximately 6,000 simulated PV plants.</li>
    <li>Solar power plant locations were determined based on the capacity expansion plan for high-penetration renewables in Phase 2 of the Western Wind and Solar Integration Study and the Eastern Renewable Generation Integration Study.</li>
    <li>NREL generated the 5-minute data set using the Sub-Hour Irradiance Algorithm. 
    <li>The day-ahead solar forecast data for locations in the western United States were generated by 3TIER based on numerical weather predication simulations for Phase 1 of the Western Wind and Solar Integration Study. 
    <li>NREL generated the day-ahead solar forecast data in eastern U.S. locations using the Weather Research and Forecasting model.</li>
    <li>The data are for specific years and should not be assumed to be representative of typical radiation levels for a site.</li>
    <li>These data should not generally be used for site-specific project development work.</li>
    <br>

### Objectives:

<ul>
    <li>Present your selected location using graphics that best convey your results for senior leadership</li>
    <li>For the location you have identified, calculate the forecasted power that a solar array would produce in megawatts</li>
    <li>Present your calculations in a graph that shows production monthly for a year</li>
    <li>What other graphics can provide insight into the forecasted power production?</li>
        <li>Extra credit ideas (you can be creative and find other ways to analyze this data):</li><br>
            <li>Plot the candidate sites on a map with an overlay of their forecasted production</li>
            <li> Automate the download, unziping and ingest of study data to increase the number of states you analyze</li>
            <li>What impact did weather have on power production rates?</li>

### Notebook Objectives:

<ul>
    <li>Create State directory structure - pull data from NREL.</li>
    <li>Create SWSD Directory Structure - Create SWSD Summary Data by state.</li>
    <li>Write SWSD_Summary_Records_Statistics Log file.</li>
</ul>

-----------------------------

### Libraries

In [1]:
import requests as req     # DOWNLOADING URL VIA URL
import zipfile             # DEALING WITH ZIP COMPRESSED FILE
from io import BytesIO     # READING FILE FROM BUFFER
import os                  # DIRECTORY CREATION
import pandas as pd        # PANDAS DATAFRAME
import datetime            # DATE AND TIME

### Variables

In [2]:
# DICTIONARY OF STATES TO BE PROCESSED
states_dict = {
    "Alabama":"al",
    "Arizona":"az",
    "California":"ca",
    "Colorado":"co",
    "Florida":"fl",
    "Maine":"me",
    "New Mexico":"nm",
    "Texas":"tx"    
}

# PULL DATA FROM NREL 
pull_data_from_nrel = False

# BASE URL OF STATE-WISE SOLAR POWER DATE
data_url = "https://www.nrel.gov/grid/assets/downloads/_-pv-2006.zip"

# PATH TO PLACE DATA
data_path = 'C://Users//bruce//Documents//Data//NRGV'

# PATH TO SUMMARY DATA FILES
smry_data_path = 'C://Users//bruce//Documents//Data//SWSD_SUMMARY_DATA'

# PULL DATA
pull_data=False

# state-wise solar power data (.csv files) from the Solar Integration Studies 
# DataType (string), Latitude (float), Longitude (float), WeatherYear (int), PVType (string), CapacityMW (int)
# TimeIntervalMin (int), LocalTime (datetime), Power(MW) (float)


swsp_features = ['LocalTime','Power(MW)','State','DataType','Latitude','Longitude','WeatherYear','PVType',\
                 'CapacityMW','TimeIntervalMin']

# SWSD SUMMARY RECORDS
swsd_summary_records_path_filename = smry_data_path + "//SWSD_Summary_Records_Statistics.txt"

### User Defined Functions (UDF)

In [3]:
def create_dir(path):
    
    # DIRECTORY IS CREATED IF IT DOESN'T CURRENTLY EXIST
    if not os.path.exists(path):
        
            # DIRECTORY IS CREATED BECAUSE IT DOESN'T CURRENTLY EXIST
            os.makedirs(path)
            
            print("The " + path + " directory was created.")
            
    else:
        print("The " + path + " directory already exists.")

In [4]:
def delete_all_files(path):
    
    # DELETE ALL FILES IN DIRECTORY
    for file in os.listdir(path):
        os.remove(os.path.join(path, file))

In [5]:
def create_dir_and_load_data(pull_data=pull_data_from_nrel):
    # https://www.geeksforgeeks.org/how-to-create-directory-if-it-does-not-exist-using-python/

    # DETERMINE IF THE data_path EXISTS, IF NOT THE DIRECTORY IS CREATED

    # ITERATE THROUGH THE STATES_DICT 
    for state, state_abrev in states_dict.items():
    
        # CREATE A VARIABLE, WHICH IS THE DATA_PATH AND THE STATE IN THE STATES_DICT 
        path_and_dir = data_path + "\\" + state_abrev

        # DIRECTORY IS CREATED IF IT DOESN'T CURRENTLY EXIST
        if not os.path.exists(path_and_dir):

            # DIRECTORY IS CREATED BECAUSE IT DOESN'T CURRENTLY EXIST
            os.makedirs(path_and_dir)
            
            if pull_data == True:
                
                # TEST FOR FILE COUNT IN DIRECTORY, PULL DATA ONLY IF DIRECTORY IS EMPTY
                if len(os.listdir(temp_path)) == 0:
                
                    # LOAD DATA
                    load_data(state, state_abrev)
        
        else:
            print("The " + state + " directory already exists.")

In [6]:
def load_data(state, state_abrev):
    
    # CREATE A VARIABLE, WHICH IS THE DATA_PATH AND THE STATE IN THE STATES_DICT 
    path_and_dir = data_path + "\\" + state_abrev
    
    # IN THE DATA_PATH VARIABLE, REPLACE THE UNDERSCORE WITH THE CURRENT VALUE IN THE STATES_DICT, CREATE VARIABLE 
    #print(data_url.replace("_",state_abrev))
    state_zip_data_url = data_url.replace("_",state_abrev)
    print("Currently working " + state_zip_data_url)
    
    # BUFFER THE DATA
    state_zip_data = req.get(state_zip_data_url)
    
    # READING FILE FROM BUFFER INTO VARIABLE
    ziped_file = zipfile.ZipFile(BytesIO(state_zip_data.content))
    
    # EXTRACT BUFFERED DATA INTO DIRECTORY
    ziped_file.extractall(path_and_dir)
    
    # PRINT STATE COMPLETED
    print("Completed downloading data for the state of " + state + ".")
    
    # CLEAR OUT VARIABLES
    del([path_and_dir,state_zip_data_url,state_zip_data,ziped_file])

In [7]:
def findall(p, s):
    # YIELDS A LIST OF ALL THE POSITION INDEX OF THE PATTERN (p) WITHIN THE STRING (s)
    i = s.find(p)
    while i != -1:
        yield i
        i = s.find(p, i+1)

In [8]:
def get_pos_index(to_find, string):
    # PRODUCES A LIST OF POSITION INDEX (ZERO BASED) OF to_find WITHIN A STRING
    return [(i, string[i:i+1]) for i in findall(to_find, string)]

In [9]:
def populate_file_metadata_variables(feature_pos_index, file_string):
    
    # DECOMPOSE FILE NAME FOR META DATA, POPULATE META DATA INTO VARIABLES 
    current_pos_in_string = 0
    current_pos_in_loop = 1
    
    # DEFINE GLOBAL SCOPE VARIABLES
    global DataType_var
    global Latitude_var
    global Longitude_var
    global WeatherYear_var
    global PVType_var
    global CapacityMW_var
    global TimeIntervalMin_var
      
    for x in feature_pos_index:
        
        for y in x:
            
            if isinstance(y, int) == True:

                if current_pos_in_loop == 1:
                    DataType_var = file_string[current_pos_in_string:y]
                elif current_pos_in_loop == 2:
                    Latitude_var = file_string[current_pos_in_string:y]
                elif current_pos_in_loop == 3:
                    Longitude_var = file_string[current_pos_in_string:y]
                elif current_pos_in_loop == 4:
                    WeatherYear_var = file_string[current_pos_in_string:y]
                elif current_pos_in_loop == 5:
                    PVType_var = file_string[current_pos_in_string:y]
                elif current_pos_in_loop == 6:
                    CapacityMW_var = file_string[current_pos_in_string:y-2]
                elif current_pos_in_loop == 7:
                    TimeIntervalMin_var = file_string[current_pos_in_string:y]


                current_pos_in_string = y + 1
                current_pos_in_loop = current_pos_in_loop + 1
            
    #print([DataType_var,Latitude_var,Longitude_var,WeatherYear_var,PVType_var,CapacityMW_var,TimeIntervalMin_var])
    return DataType_var,Latitude_var,Longitude_var,WeatherYear_var,PVType_var,CapacityMW_var,TimeIntervalMin_var

### Create State Data Directory Structures and Pull Data From National Renewable Energy Laboratory (NREL)

In [10]:
create_dir_and_load_data(pull_data=False)

The Alabama directory already exists.
The Arizona directory already exists.
The California directory already exists.
The Colorado directory already exists.
The Florida directory already exists.
The Maine directory already exists.
The New Mexico directory already exists.
The Texas directory already exists.


### Create SWSD Directory Structure

In [11]:
# CREATE SWSD DATA SUMMARY DIRECTORY
create_dir(smry_data_path)
SWSD = pd.DataFrame(columns=swsp_features)

The C://Users//bruce//Documents//Data//SWSD_SUMMARY_DATA directory already exists.


### Read data - Create SWSD Summary Data by state 

In [12]:
# ITERATE states_dict
for state, state_abv in states_dict.items():
    
    # SET STATE VARIABLE TO 
    State_var = state_abv.upper()
    
    print("The process of Summary Data has start for the state of {}.".format(State_var))
    
    start_time = datetime.datetime.now()
    
    # READ FILENAMES IN STATE DIRECTORY
    dir_list = os.listdir(data_path + '\\' + state_abv)
    
    csv_count = len(dir_list)

    # 1) ITERATE THROUGH FILES (file_string) IN DIRECTORY (dir_list)
    #   2) DECOMPOSE FILE NAME (file_string) FOR META DATA, POPULATE META DATA INTO VARIABLES
    #      3) POPULATE DATAFRAME WITH FILE_NAME META DATA VARIABLES AND FILE CONTENTS
    #         4) AT THE CONCLUSION OF PROCESSING ALL THE CSV FILES FOR A STATE, WRITE A SUMMARY FILE

    # 1) ITERATE THROUGH FILES (file_string) IN DIRECTORY (dir_list)
    for swsd_csv_file_string in dir_list:
        
        
        #    2) DECOMPOSE FILE NAME (swsd_csv_file_string) FOR META DATA, POPULATE META DATA INTO VARIABLES
        feature_pos_index = get_pos_index('_', swsd_csv_file_string)

        DataType_var,Latitude_var,Longitude_var,WeatherYear_var,PVType_var,CapacityMW_var,\
            TimeIntervalMin_var = populate_file_metadata_variables(feature_pos_index, swsd_csv_file_string)
        
        #   3) POPULATE DATAFRAME WITH FILE_NAME META DATA VARIABLES AND FILE CONTENTS
        
        # SUBSTANTIATE path_filename WITH THE CURRENT DIRECTORY AND CSV FILE NAME
        path_filename = data_path + '\\' + State_var + '\\' + swsd_csv_file_string
        
        # READ CSV
        temp_df = pd.read_csv(path_filename, header=0)
        
        # POPULATE FEATURES BASED UPON CURRENT VARIABLES
        temp_df['State'] = State_var
        temp_df['DataType'] = DataType_var
        temp_df['Latitude'] = Latitude_var
        temp_df['Longitude'] = Longitude_var
        temp_df['WeatherYear'] = WeatherYear_var
        temp_df['PVType'] = PVType_var
        temp_df['CapacityMW'] = CapacityMW_var
        temp_df['TimeIntervalMin'] = TimeIntervalMin_var
        
        #print(temp_df.head())
        
        # APPEND temp_df TO SWSD DATAFRAM
        SWSD = SWSD.append(temp_df)
        
        
    # 4) AT THE CONCLUSION OF PROCESSING ALL THE CSV FILES FOR A STATE, WRITE A SUMMARY FILE
    
    stop_time = datetime.datetime.now()
    processing_elapsed_time = stop_time - start_time
    
    # SUBSTANTIATE SUMMARY FILE: path_filename WITH THE CURRENT DIRECTORY AND CSV FILE NAME
    smry_data_path_filename = smry_data_path + "\\" + State_var + '_SWSD_SUMMARY_DATA'

    # PROCESSING STATISTICS
    start_time_txt = "The process started at: {}".format(start_time)
    stop_time_txt = "The process finished at: {}".format(stop_time)
    processing_elapsed_time_txt = "The processing elapsed time was: {}".format(processing_elapsed_time)
    state_name_txt = "The State being processed is: {}.".format(State_var)
    csv_cnt_txt = "The total number of csv files to be processed is: {:,}".format(csv_count)
    state_smry_file_txt = "The {}_SWSD_SUMMARY_DATA has been written.".format(State_var)
    smry_record_cnt_txt = "There are {:,} records in the file.".format(str(len(SWSD)))
    new_line = '\n'

    
    # WRITE CSV
    SWSD.to_csv(smry_data_path_filename +'.csv', header=True, index=False)
    
    # PRINT PROCESSING STATISTICS TO SCREEN
    print(start_time_txt,new_line,stop_time_txt,new_line,processing_elapsed_time_txt,new_line,state_name_txt,\
          new_line,csv_cnt_txt,new_line,smry_record_cnt_txt,new_line,state_smry_file_txt,new_line)
    
    # CAPTURE PROCESSING STATISTICS FOR TXT FILE
    data_reduction = [start_time_txt,stop_time_txt,processing_elapsed_time_txt,state_name_txt,csv_cnt_txt,\
                      state_smry_file_txt,smry_record_cnt_txt,"\n"]
    
    # MEMORALIZE PROCESSING STATISTICS
    with open(swsd_summary_records_path_filename, 'a') as new_file:
        for line in data_reduction:
            new_file.write(line)
            new_file.write('\n')
    
    # CLEAR SWSD FILE OF ALL RECORDS
    SWSD = SWSD[0:0]

The process of Summary Data has start for the state of AL.
The process started at: 2022-10-16 06:42:52.173001 
 The process finished at: 2022-10-16 07:05:28.366052 
 The processing elapsed time was: 0:22:36.193051 
 The State being processed is: AL. 
 The total number of csv files to be processed is: 411 
 There are 16801680 records in the file. 
 The AL_SWSD_SUMMARY_DATA has been written. 

The process of Summary Data has start for the state of AZ.
The process started at: 2022-10-16 07:06:06.486762 
 The process finished at: 2022-10-16 07:41:48.187274 
 The processing elapsed time was: 0:35:41.700512 
 The State being processed is: AZ. 
 The total number of csv files to be processed is: 513 
 There are 20971440 records in the file. 
 The AZ_SWSD_SUMMARY_DATA has been written. 

The process of Summary Data has start for the state of CA.
The process started at: 2022-10-16 07:42:37.098666 
 The process finished at: 2022-10-16 11:00:09.074094 
 The processing elapsed time was: 3:17:31.975